In [1]:
from glob import glob
from tqdm import tqdm
import json

In [2]:
files = glob('*-00000-of-00001.jsonl')
files

['cpp-00000-of-00001.jsonl',
 'scala-00000-of-00001.jsonl',
 'c-sharp-00000-of-00001.jsonl',
 'shell-00000-of-00001.jsonl',
 'go-00000-of-00001.jsonl',
 'python-00000-of-00001.jsonl',
 'pythonanalytics-00000-of-00001.jsonl',
 'sql-00000-of-00001.jsonl',
 'java-00000-of-00001.jsonl',
 'dockerfile-00000-of-00001.jsonl',
 'rust-00000-of-00001.jsonl',
 'javascript-00000-of-00001.jsonl',
 'typescript-00000-of-00001.jsonl',
 'python-analytics-00000-of-00001.jsonl',
 'csharp-00000-of-00001.jsonl',
 'ruby-00000-of-00001.jsonl',
 'php-00000-of-00001.jsonl',
 'swift-00000-of-00001.jsonl',
 'cuda-00000-of-00001.jsonl',
 'kotlin-00000-of-00001.jsonl']

In [3]:
filtered = []
rejected = []

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if 'code snippet' in l['instruction']:
                rejected.append(l)
                continue
            answer = l['answer'].split('\n\n')
            if len(answer[0]) < 20:
                answer = answer[1:]
            answer = '\n\n'.join(answer)
            if len(answer.split()) < 100:
                continue
            filtered.append((l['instruction'], answer))
        print(f, len(filtered))
len(filtered)

10747it [00:00, 55703.11it/s]


cpp-00000-of-00001.jsonl 8119


10017it [00:00, 50925.98it/s]


scala-00000-of-00001.jsonl 16832


5693it [00:00, 62357.64it/s]


...

13843it [00:00, 64574.69it/s]


cuda-00000-of-00001.jsonl 211002


12790it [00:00, 62362.56it/s]

kotlin-00000-of-00001.jsonl 219018


219018

In [4]:
import json
mapping = {}

for f in glob('/home/husein/ssd3/ctranslate2/postfilter-wizardcoder.texts*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            mapping[l['src']] = l['r']
len(mapping)

373081

In [5]:
with open('postfilter.jsonl', 'w') as fopen:
    for l in filtered:
        d = {
            'instruction': l[0],
            'instruction_ms': mapping.get(l[0]),
            'answer': l[1],
            'answer_ms': mapping.get(l[1]),
        }
        fopen.write(f'{json.dumps(d)}\n')

In [6]:
!tail -n 3 postfilter.jsonl

{"instruction": "We have a `DialogFragment` named `AlimentAddDialogFragment` that allows users to add an `Aliment` item to a list. The `Aliment` class contains a list of `AlimentState` objects, and each `AlimentState` has several `AlimentStateMeasure` objects associated with it. These `AlimentStateMeasure` objects contain a `measure` and a `quantity`.\n\nWhen a user selects an `AlimentState`, the dialog shows a spinner with the names of all measures associated with the selected `AlimentState`, and a quantity `EditText`. The user can choose a measure and input a quantity to be associated with the selected `AlimentState`. The dialog then calls the `onClick` function of its `Listener` interface, which takes the `AlimentState` and the computed quantity as parameters.\n\nThe problem arises when the user inputs a decimal quantity instead of an integer. The current implementation truncates the decimal part and the user loses the information. Your task is to modify the `AlimentAddDialogFragmen

In [7]:
from huggingface_hub import HfApi
api = HfApi()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [8]:
api.upload_file(
    path_or_fileobj='postfilter.jsonl',
    path_in_repo='postfilter.jsonl',
    repo_id='mesolitica/mixtral-wizardcoder',
    repo_type='dataset',
)

postfilter.jsonl:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/mixtral-magicoder/commit/9b8b5b071273e3285f097021f8b91cb74f0aa8a1', commit_message='Upload postfilter.jsonl with huggingface_hub', commit_description='', oid='9b8b5b071273e3285f097021f8b91cb74f0aa8a1', pr_url=None, pr_revision=None, pr_num=None)